YOLO V3
===

# 1.概述

YOLO算法创新性地提出了将输入图片进行N*N的栅格化（每个小单元叫grid cell），然后将图片中某个对象的位置的预测任务交与该对象中心位置所在的grid cell的bouding box。简单理解的话，可以认为这也是一种很粗糙的区域推荐（region proposal），在训练的时候，我们通过grid cell的方式告诉模型，图片中对象A应该是由中心落在特定grid cell 的某个范围内的某些像素组成，模型接收到这些信息后就在grid cell周围以一定大小范围去寻找所有满足对象A特征的像素，经过很多次带惩罚的尝试训练后，它就能找到这个准确的范围了（说明不是瞎找，如滑动窗口），当然这个方位不仅是指长宽的大小范围，也包括小幅度的中心位置坐标变化，但是不管怎么变，中心位置不能越过该grid cell的范围。这大大限制了模型在图片中瞎找时做的无用功。这样将位置检测和类别识别结合到一个CNN网络中预测，即只需要扫描一遍（you only look once）图片就能推理出图片中所有对象的位置信息和类别。举例如下图。

![Images](images/02_04_001.jpg)

不管是YOLOv1还是v2、v3，其主要的核心还是以上所述，只是在bounding box的拟合方式、骨干网络的设计、模型训练的稳定性、精度方面有所提升罢了

# 2.训练
YOLO算法几乎是输入原图就直接预测出每个grid cell“附近”是否有某个对象和具体的box位置，那最终这个想法数学化后便体现在loss函数上。在提出loss函数之前要先了解三个概念
- anchor box
- 置信度(confidence)
- 对象条件类别概率(conditional class probabilities)

在网络最后的输出中，对于每个grid cell对应bounding box的输出有三类参数
- 对象的box参数，一共是四个值，即box的中心点坐标（x,y）和box的宽和高（w,h）
- 置信度，这是个区间在\[0,1\]之间的值
- 一组条件类别概率，都是区间在\[0,1\]之间的值，代表概率。

## 2.1.Anchor box

### 2.1.1.Anchor box的由来
anchor box(也称为bounding box prior)其实就是从训练集的所有ground truth box中统计出来的在训练集中最经常出现的几个box形状和尺寸，统计的方法就是k-means算法。比如，在某个训练集中最常出现的box形状有扁长的、瘦高的和宽高比例差不多的正方形这三种形状。我们可以预先将这些统计上的先验经验加入到模型中，这样模型在学习的时候，瞎找的可能性就更小了些，当然就有助于模型快速收敛了。

以前面提到的训练数据集中的ground truth box最常出现的三个形状为例，当模型在训练的时候我们可以告诉它，你要在grid cell 1附近找出的对象的形状要么是扁长的、要么是瘦高的、要么是长高比例差不多的正方形，你就不要再瞎试其他的形状了。anchor box其实就是对预测的对象范围进行约束，并加入了尺寸先验经验，从而可以有效解决对象多尺度的问题

当我们只对图片中一个对象（且图片中只有一个对象）进行box回归时，我们只需要一个box回归器，但是当我们对图片中多个对象进行回归时（甚至一个类别会有多个对象），这时使用多个box回归器预测多个对象位置时就会发生冲突，因为每个预测器都可能不受约束地预测图片中任何一个对象的位置和类别。这时，我们就可以使用anchor来对每个回归器进行约束，只让每个回归器负责一块独立区域内的对象box回归。以YOLO算法举例，每个grid cell的位置其实也可以看做是anchor的位置（这不同于SSD或者Faster RCNN的anchor），如果最终的输出为13x13，也即有13x13个grid cell，每个grid cell有三个anchor的话，整个模型就有13x13x3个回归器，每个回归器只负责相应grid cell附近的对象预测。

## 2.2.置信度(confidence)

## 2.3.对象条件类别概率(conditional class probabilities)

## 2.4.损失函数

$$
\begin{eqnarray*}
Loss&=&\sum_{i=0}^{S^2}\sum_{j=0}^B\phi_{ij}^{obj}\{[\sigma(t_x)_i^j-\sigma(\hat{t}_x)_i^j]^2+[\sigma(t_y)_i^j-\sigma(\hat{t}_y)_i^j]^2\} \tag{1} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\phi_{ij}^{obj}\{[{t_w}_i^j-{\hat{t}_w}_i^j]^2+[{t_h}_i^j-{\hat{t}_h}_i^j]^2\} \tag{2} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\rho_{ij}(C_i^j-\hat{C}_i^j)^2 \tag{3} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\sum_{c \in classes}\phi_{ij}^{obj}[p_i^j(c)-\hat{p}_i^j(c)]^2 \tag{4}
\end{eqnarray*}
$$

### 2.4.1.$\phi_{ij}^{obj}$

### 2.4.2.$\rho_{ij}$

### 2.4.3.$S$

## 2.5.跨尺寸预测(Predictions across scales)
YOLO算法从三个不同的尺寸预测对象box，这三个不同的尺寸来自不同层级的卷积层的输出。该方法借鉴了feature pyramid network的思想: 由于卷积层每隔几层，特征映射(feature mapping)的宽和高就会减少，而通道数会增加，随着网络层次的加深，特征映射组成的形状类似于金字塔，如果将不同层级的特征映射转换为最终的输出，那么将有助于提升模型在对象不同尺度大小上的表现，即有助于提高模型从小目标到大目标的综合检测（box的精度）能力，

![Images](images/02_04_002.jpg)

- 从网络的不同层次映射不同尺寸的输出，如图中从79层（外加两个卷积层）得到13*13的的输出;从91层（外加两个卷积层）得到26*26的输出;最后再得到52*52的输出。
- 后面的高层结合使用低层特征（图中的86、98层，分别使用了61层和36层的特征映射），使高层能使用细粒度（fine grained）特征和更多的语义信息。
- 最后一个尺寸输出使用了前两个尺寸计算的特征映射，使得最后的尺寸输出也能使用细粒度。
- 每个YOLO输出层中，每个grid cell的bounding box数量为3，而不是9，这样不同的YOLO输出层便能负责不同尺寸大小的对象预测了，这个思想来自SSD。例如，COCO数据集中，作者让YOLO scale1负责预测的尺寸有(116,90)、(156,198)和(373,326), YOLO scale2负责预测的尺寸有(30,61)、(62,45)和(59,119)，YOLO scale3负责预测的尺寸有(10,13)、 (16,30)和 (33,23)